# **Tugas Individu Modul 3 Praktikum ABD**
Nama : Claudhea Angeliani

NIM : 121450124

Kelas : RB

### Instalasi pyspark
pyspark atau Apache Spark adalah Python API yang digunakan untuk proses data berskala besar *(Big Data)* yang *realtime*.

In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=96d51616d52b16370598dd7f065356cd6e387155fe9a715c5997f895b9059888
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


### Import Library

Pada pemrosesan *big data* ini, digunakan 3 *library* yaitu numpy, pandas dan ast.

*   Numpy digunakan untuk proses manipulasi dan analisis data dalam bentuk numerik, array, matriks dan fungsi matematika.
*   Pandas digunakan untuk memanipulasi data berbasis tabel atau *dataframe*
*   ast atau *Abstract Syntax Tree* digunakan untuk membantu menganalisis, mengubah, dan mengoptimalkan kode Python.

In [2]:
# Mengimport library numpy, pandas, ast(Abstract Syntax Trees)
import numpy as np
import pandas as pd
import ast

### Dataset

Dataset yang digunakan pada modul ini adalah movie data part 1. File berformat csv dan di baca dengan kode `spark.read`. Lalu ditampilkan skemanya dengan `printSchema` lalu mencetak dataframnya dengan `df.show(20)` artinya menampilkan 20 baris paling atas dari dataset.

Dataset ini berisi tentang berbagai judul film beserta metadatanya. Metadata tersebut dibuat menjadi tabel dengan variabel sebagai berikut:

*   belongs_to_collections: kategori apakah film tersebut adalah koleksi dari sebuah produksi film series atau koleksi khusus
*   budget: biaya yang dihabiskan untuk produksi film
*   id: nomor registrasi film
*   original_language: bahasa asli yang digunakan dalam film
*   original_title: title asli/terdahulu
*   overview: sinopsis atau rangkuman pendek film
*   popularity : nilai kumulative dari jumlah rating, votes, view, dsb.
*   production_companies : company atau perusahaan yang memproduksi film
*   production_countries: asal negara perusahaan yang memproduksi film
*   release_data : tanggal rilis film
*   revenue: nilai keuntungan yang dihasilkan saat film tayang
*   runtime: panjang durasi film
*   status: keterangan atau status film
*   tagline: kalimat ikonik yang digunakan untuk promosi film
*   title: judul final film
*   vote_averange: jumlah rata-rata vote yang didapatkan film. Vote ini biasanya didapatkan dari penilaian penonton.

**Tujuan** dibuatnya dataset ini adalah mendapatkan informasi penting terkait preferensi penonton, budget hingga teknis perfilman, sehingga rumah produksi dapat merancang produksi film dengan lebih strategis dan efisien. Hal ini menanggulangi terciptanya film-film yang overbudget namun revenue atau keuntungannya tidak sebanyak biaya awalnya.

In [3]:
from pyspark.sql import SparkSession

# Inisialisasi SparkSession
spark = SparkSession.builder \
    .appName("Create Spark DataFrame") \
    .getOrCreate()

# Memuat file CSV ke Pandas DataFrames
df = spark.read.options(header='True', inferSchema='True', delimiter='|') \
    .csv('/content/movie_data_part1.csv')
df.printSchema()

# Menampilkan beberapa baris pertama dari Spark DataFrame
df.show(20)

root
 |-- belongs_to_collection: string (nullable = true)
 |-- budget: double (nullable = true)
 |-- id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: double (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- revenue: double (nullable = true)
 |-- runtime: integer (nullable = true)
 |-- status: string (nullable = true)
 |-- tagline: string (nullable = true)
 |-- title: string (nullable = true)
 |-- vote_average: double (nullable = true)

+---------------------+---------+-----+-----------------+--------------------+--------------------+------------------+--------------------+--------------------+------------+---------+-------+--------+--------------------+--------------------+------------+
|belongs_to_collection|   budget|   id|original_lan

### Eksplorasi Data

Setelah itu, kita melakukan eksplorasi data. Pertama, kita lihat tipe data dari masing-masing kolom. Ada beberapa tipe, diantaranya: string yaitu tipe data berupa kumpulan karakter, simbol atau nomor yang diperlakukan sebagai teks. Double adalah tipe data pecahan dengan tingkat ketelitian secara ganda atau double precision dengan menggunakan ruang penyimpanan 64-bit dalam menyimpan nilainya. Sedangkan integer adalah tipe data bilangan bulat atau numerik umum. Berikut kategori tipe data beserta kolomnya.
*  String: belongs_to_collections, id, original_language, original_title, overview, production_companies. production_countries, release_date, status, dan title
*  Double: budget, popularity, revenue dan vote_average
*  Integer: runtime

In [4]:
# Memeriksa tipe data dari setiap kolom dalam dataframe
df.dtypes

[('belongs_to_collection', 'string'),
 ('budget', 'double'),
 ('id', 'string'),
 ('original_language', 'string'),
 ('original_title', 'string'),
 ('overview', 'string'),
 ('popularity', 'double'),
 ('production_companies', 'string'),
 ('production_countries', 'string'),
 ('release_date', 'string'),
 ('revenue', 'double'),
 ('runtime', 'int'),
 ('status', 'string'),
 ('tagline', 'string'),
 ('title', 'string'),
 ('vote_average', 'double')]

Lalu, digunakan `df.count` untuk menghitung jumlah baris pada tiap kolom df. Didapatkan bahwa ada 43.998 baris dengan nilai NULL. Disini saya tidak mereduksi data dalam dataframe ini, karena jika direduksi dengan menghilangkan tiap baris dengan nilai NULL, hanya tersisa 2000an baris saja, sehingga sudah tidak bisa terbilang big data. Maka dari itu sini tetap digunakan data semula.

In [5]:
# Menghitung jumlah baris yang tidak mengandung nilai null dalam setiap kolom
df.count()

43998

Dengan didapatkan 43.998 data, maka data sejumlah itulah yang akan direcord atau dianggap sebagai dataset yang akan dieksplorasi dengan Spark.

In [6]:
# Menampilkan total jumlah record dalam dataset movie
print('The total number of records in the movie dataset is ' +str(df.count()))

The total number of records in the movie dataset is 43998


Dari banyak kolom yang ada, kita memilih beberapa kolom untuk ditampilkan dan dieksplorasi. Disini dipilih kolom id, budget, popularity, release_date, revenue dan title dengan `select_colums = ['coloumns']` lalu menyimpan kolom yang dipilih dalam dataset df. Kemudian dataframe tersebut ditampilkan sebnayak 20 baris pertama.



In [7]:
# Memilih kolom-kolom yang akan ditampilkan
select_columns=['id','budget','popularity','release_date','revenue','title']
df=df.select(select_columns)
# Menampilkan 20 baris pertama dari dataframe
df.show()

+-----+---------+------------------+------------+---------+--------------------+
|   id|   budget|        popularity|release_date|  revenue|               title|
+-----+---------+------------------+------------+---------+--------------------+
|43000|      0.0|             2.503|  1962-05-23|      0.0|The Elusive Corporal|
|43001|      0.0|              5.51|  1962-11-12|      0.0|  Sundays and Cybele|
|43002|      0.0|              5.62|  1962-05-24|      0.0|Lonely Are the Brave|
|43003|      0.0|             7.159|  1975-03-12|      0.0|          F for Fake|
|43004| 500000.0|             3.988|  1962-10-09|      0.0|Long Day's Journe...|
|43006|      0.0|             3.194|  1962-03-09|      0.0|           My Geisha|
|43007|      0.0|             2.689|  1962-10-31|      0.0|Period of Adjustment|
|43008|      0.0|             6.537|  1959-03-13|      0.0|    The Hanging Tree|
|43010|      0.0|             4.297|  1962-01-01|      0.0|Sherlock Holmes a...|
|43011|      0.0|           

Kode ini menampilkan output yang sama dengan kode diatas, hanya saja lebih tingkas dan tidak menyimpan dataset baru ke dataset df.

In [ ]:
# Menampilkan kolom yang dipilih dari data frame
df.select('id','budget','popularity','release_date','revenue','title').show()

+-----+---------+----------+------------+---------+--------------------+
|   id|   budget|popularity|release_date|  revenue|               title|
+-----+---------+----------+------------+---------+--------------------+
|43000|      0.0|     2.503|  1962-05-23|      0.0|The Elusive Corporal|
|43001|      0.0|      5.51|  1962-11-12|      0.0|  Sundays and Cybele|
|43002|      0.0|      5.62|  1962-05-24|      0.0|Lonely Are the Brave|
|43003|      0.0|     7.159|  1975-03-12|      0.0|          F for Fake|
|43004| 500000.0|     3.988|  1962-10-09|      0.0|Long Day's Journe...|
|43006|      0.0|     3.194|  1962-03-09|      0.0|           My Geisha|
|43007|      0.0|     2.689|  1962-10-31|      0.0|Period of Adjustment|
|43008|      0.0|     6.537|  1959-03-13|      0.0|    The Hanging Tree|
|43010|      0.0|     4.297|  1962-01-01|      0.0|Sherlock Holmes a...|
|43011|      0.0|     4.417|  1962-01-01|      0.0|  Sodom and Gomorrah|
|43012|7000000.0|     4.722|  1962-11-21|4000000.0|

Kode ini menampilkan records kolom kedua pada dataframe yaitu popularity dengan `df.select(df[2]).show()`.

In [8]:
# Menampilkan kolom kedua(popularity) dari data frame
df.select(df[2]).show()

+------------------+
|        popularity|
+------------------+
|             2.503|
|              5.51|
|              5.62|
|             7.159|
|             3.988|
|             3.194|
|             2.689|
|             6.537|
|             4.297|
|             4.417|
|4.7219999999999995|
|             2.543|
|             4.303|
|             3.493|
|             2.851|
|             4.047|
|             2.661|
|             3.225|
|              5.72|
|             3.292|
+------------------+
only showing top 20 rows



Kode `df.show(25, False)` akan menampilkan records dari DataFrame df dengan menggunakan metode show dan menampilkan 25 baris data dengan pengaturan 'False' untuk tidak memotong data.

In [9]:
# Menampilkan 25 baris pertama dari dataframe
df.show(25, False)

+-----+---------+------------------+------------+---------+--------------------------------------------------------+
|id   |budget   |popularity        |release_date|revenue  |title                                                   |
+-----+---------+------------------+------------+---------+--------------------------------------------------------+
|43000|0.0      |2.503             |1962-05-23  |0.0      |The Elusive Corporal                                    |
|43001|0.0      |5.51              |1962-11-12  |0.0      |Sundays and Cybele                                      |
|43002|0.0      |5.62              |1962-05-24  |0.0      |Lonely Are the Brave                                    |
|43003|0.0      |7.159             |1975-03-12  |0.0      |F for Fake                                              |
|43004|500000.0 |3.988             |1962-10-09  |0.0      |Long Day's Journey Into Night                           |
|43006|0.0      |3.194             |1962-03-09  |0.0      |My Ge

Kode ini melibatkan modul pyspark yaitu isnan, isnull, when dan count. Tujuannya adalah untuk menghitung jumlah nilai null dalam kolom popularity. Hasilnya, jumlah NULL atau missing value pada kolom popularity adalah 0. Hanya saja, nilai dari budget ini kebanyakan nilai 0.00, yang mana nilai itu tidak mungkin untuk membuat sebuat film. Hal ini menjadi anomali dalam dataset ini.

In [10]:
# Menghitung jumlah nilai null dalam kolom 'popularity'

from pyspark.sql.functions import isnan, isnull, when, count
missing_values_count = df.select(count(when(isnull(df['popularity'])|(isnan(df['popularity'])) | (df['popularity'] == ''), df['popularity'])).alias('missing_values')).collect()[0]['missing_values']

print("Number of missing value in the 'popularity' column:", missing_values_count)

Number of missing value in the 'popularity' column: 0


DIlakukan hal yang sama pada kolom budget dengan hasil missing valuenya 0.

In [11]:
# Menghitung jumlah nilai null dalam kolom 'budget'

from pyspark.sql.functions import isnan, isnull, when, count
missing_values_count = df.select(count(when(isnull(df['revenue'])|(isnan(df['revenue'])) | (df['revenue'] == ''), df['revenue'])).alias('missing_values')).collect()[0]['missing_values']

print("Number of missing value in the 'revenue' column:", missing_values_count)

Number of missing value in the 'revenue' column: 0


Pada proses yang menggunakan `.groupBy` akan melakukan pengelompokkan dataframe, disini dilakukan pengelompokkan berdasarkan judul beserta jumlahnya. Lalu ditampilkan 20 judul teratas berserta jumlahnya dengan `.show()`.

In [16]:
# Mengelompokkan dataframe berdasarkan judul dan menghitung jumlahnya
df.groupBy(df['title']).count().show()

+--------------------+-----+
|               title|count|
+--------------------+-----+
|   The Corn Is Green|    1|
|Meet The Browns -...|    1|
|Morenita, El Esca...|    1|
| Father Takes a Wife|    1|
|The Werewolf of W...|    1|
|My Wife Is a Gang...|    1|
|Depeche Mode: Tou...|    1|
|  A Woman Is a Woman|    1|
|History Is Made a...|    1|
|      Colombian Love|    1|
|        Ace Attorney|    1|
|     Not Like Others|    1|
|40 Guns to Apache...|    1|
|          Middle Men|    1|
|         It's a Gift|    1|
|    La Vie de Bohème|    1|
|Rasputin: The Mad...|    1|
|The Ballad of Jac...|    1|
|         How to Deal|    1|
|             Freaked|    1|
+--------------------+-----+
only showing top 20 rows



Hampir sama dengan kode diatas, namun disini menggunakan modul pyspark untuk mengimport desc. Modul ini akan mengurutkan judul dari yang terbanyak. Namun didapatkan bahwa yang terbanyak adalah NULL sebanyak 304. Judul NULL ini muncul karena di awal,

In [17]:
# Mengelompokkan dataframe berdasarkan judul, menghitung jumlahnya, dan mengurutkannya dari yang terbanyak
from pyspark.sql.functions import desc
df.groupby(df['title']).count().sort(desc("count")).show(10, False)

+--------------------+-----+
|title               |count|
+--------------------+-----+
|NULL                |304  |
|Les Misérables      |8    |
|The Three Musketeers|8    |
|Cinderella          |8    |
|A Christmas Carol   |7    |
|The Island          |7    |
|Dracula             |7    |
|Hamlet              |7    |
|Frankenstein        |7    |
|Cleopatra           |6    |
+--------------------+-----+
only showing top 10 rows



Pada sesi kali ini masih menggunakan modul pyspark yaitu col. Col ini akan mengembalikan nilai kolom berdasarkan nama. Kode dibawah melakukan filtering untuk menghilangkan baris dengan nilai kosong pada kolom title. Hal ini dilakukan karena nilai NULL adalah judul dengan jumlah terbanyak yaitu 304.
Setelah difilter, lalu dataframe dikelompokkan berdasarkan judul, jumlah dan menampilkan hasil yang memiliki jumlah lebih dari 4. Didapatkan 20 baris judul dengan Les Misérables sebagai kata paling banyak dipakai, yaitu sejumlah 8.

In [18]:
from pyspark.sql.functions import col

# Filter dataframe untuk menghilangkan baris dengan nilai kosong atau null pada kolom 'title'
df_temp = df.filter((df['title'] !='') & (df['title'].isNotNull()) &
  (~isnan(df['title'])))

# Mengelompokkan dataframe yang telah difilter berdasarkan judul, menghitung jumlahnya, dan memfilter hasilnya yang memiliki jumlah lebih dari 4
df_temp.groupby(df_temp['title']).count().filter("`count` > 4").sort(col("count").desc()).show(10, False)

+--------------------+-----+
|title               |count|
+--------------------+-----+
|Les Misérables      |8    |
|The Three Musketeers|8    |
|Cinderella          |8    |
|A Christmas Carol   |7    |
|The Island          |7    |
|Frankenstein        |7    |
|Dracula             |7    |
|Hamlet              |7    |
|Treasure Island     |6    |
|The Lost World      |6    |
+--------------------+-----+
only showing top 10 rows



Masih dengan modul pyspark import col, disini menghitung jumlah judul atau kata yang digunakan dalam judul yang muncul lebih dari 4 kali. Hasilnya, ada 111 kata yang diulang sebanyak lebih dari 4 kali.

In [19]:
from pyspark.sql.functions import col

# Menghitung jumlah judul yang muncul empat kali atau lebih
jumlah_judul = df_temp.groupby(df_temp['title']).count().filter("`count` >= 4").count()

# Menampilkan jumlah judul yang diulang empat kali atau lebih
print("Jumlah judul yang diulang empat kali atau lebih:", jumlah_judul)

Jumlah judul yang diulang empat kali atau lebih: 111


Menghapus dataframe sementara df_temp dengan fungsi `del`

In [ ]:
del df_temp

Kita kembali memakai dataset df. Disini kembali di cek tipe data dari tiap kolom terpilih pada dataset.

In [ ]:
# Menampilkan tipe data kolom dalam dataframe sebelum konversi
df.dtypes

[('id', 'string'),
 ('budget', 'double'),
 ('popularity', 'double'),
 ('release_date', 'string'),
 ('revenue', 'double'),
 ('title', 'string')]

Selanjutnya, kolom budget yang semula bertipe 'double' akan diubah menjadi bentuk float dengan fungsi `.cast("tipe data")`.

In [ ]:
# Mengonversi kolom 'budget' menjadi tipe data float
df = df.withColumn('budget', df['budget'].cast("float"))

Dibawah ini merupakan hasil dari konversi tipe data sebelumnya

In [ ]:
# Menampilkan tipe data kolom dalam dataframe setelah konversi
df.dtypes

[('id', 'string'),
 ('budget', 'float'),
 ('popularity', 'double'),
 ('release_date', 'string'),
 ('revenue', 'double'),
 ('title', 'string')]

Penyesuaian tipe data tidak hanya dibutuhkan oleh budget, namun beberapa variabel juga harus dikonversi ke bentuk tipe data yang benar. Jika dilakukan satu persatu seperti kode sebelumnya, akan memakan banyak waktu. Maka dari itu, digunakan fungsi looping untuk konversi tipe data.

Pertama, akan dikelompokkan kolom-kolom yang termasuk integer, float dan date. Setelah itu dimasukkan ke looping sesuai dengan tipe data. Setelah itu, dicek kembali apakah datanya sudah benar. Setelah dibetulkan, berikut hasilnya :
 * id : string --> integer
 * budget: float --> float
 * popularity: double --> float
 * release_date: string --> date
 * revenue: double --> float
 * title: string --> string

In [ ]:
# Import library yang dibutuhkan
from pyspark.sql.types import *

# Menentukan variabel integer, float, dan date
int_vars = ['id']
float_vars = ['budget', 'popularity', 'revenue']
date_vars = ['release_date']

# Loop untuk mengonversi tipe data kolom sesuai dengan variabel yang ditentukan
for column in int_vars:
  df = df.withColumn(column, df[column].cast(IntegerType()))

for column in float_vars:
  df = df.withColumn(column, df[column].cast(FloatType()))

for column in date_vars:
  df = df.withColumn(column, df[column].cast(DateType()))

# Menampilkan tipe data kolom dalam dataframe setelah pengonversian
df.dtypes

[('id', 'int'),
 ('budget', 'float'),
 ('popularity', 'float'),
 ('release_date', 'date'),
 ('revenue', 'float'),
 ('title', 'string')]

Menampilkan 10 baris pertama dataframe setelah konversi tipe data.

In [ ]:
# Menampilkan 10 baris pertama dari dataframe
df.show(10, False)

+-----+--------+----------+------------+-------+---------------------------------------+
|id   |budget  |popularity|release_date|revenue|title                                  |
+-----+--------+----------+------------+-------+---------------------------------------+
|43000|0.0     |2.503     |1962-05-23  |0.0    |The Elusive Corporal                   |
|43001|0.0     |5.51      |1962-11-12  |0.0    |Sundays and Cybele                     |
|43002|0.0     |5.62      |1962-05-24  |0.0    |Lonely Are the Brave                   |
|43003|0.0     |7.159     |1975-03-12  |0.0    |F for Fake                             |
|43004|500000.0|3.988     |1962-10-09  |0.0    |Long Day's Journey Into Night          |
|43006|0.0     |3.194     |1962-03-09  |0.0    |My Geisha                              |
|43007|0.0     |2.689     |1962-10-31  |0.0    |Period of Adjustment                   |
|43008|0.0     |6.537     |1959-03-13  |0.0    |The Hanging Tree                       |
|43010|0.0     |4.297

Pada sesi ini, digunakan Spark Session untuk melakukan analisis. Proses analisis ini meliputi pembuatan summary/kesimpulan dari dataframe, isinya adalah count(jumlah data), mean(rata-rata data), standar deviasi(nilai/jarak persebaran data), min(nilai terendah) dan max(nilai tertingg). Disini title memiliki summary yang aneh karena title bertipe data non-numerik yaitu string, sehingga tidak dapat memunculkan statistik yang diinginkan.

In [ ]:
# Import library yang dibutuhkan
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
# Membuat sesi Spark
spark = SparkSession.builder \
    .appName("Data Analysis") \
    .getOrCreate()
# Membuat summary statistik dari datafram
summary_df = df.describe()
summary_df.show()

+-------+------------------+--------------------+------------------+-------------------+--------------------+
|summary|                id|              budget|        popularity|            revenue|               title|
+-------+------------------+--------------------+------------------+-------------------+--------------------+
|  count|             43784|               43873|             43783|              43783|               43694|
|   mean|44502.304312077475|  3736901.8349631676| 5.295444259579189|  9697079.597382545|            Infinity|
| stddev|  27189.6465886264|1.5871814952777285E7|6.1680305192082665|5.687938449628816E7|                 NaN|
|    min|                 2|                 0.0|               0.6|                0.0|!Women Art Revolu...|
|    max|            100988|               3.8E8|             180.0|       2.78796518E9|       시크릿 Secret|
+-------+------------------+--------------------+------------------+-------------------+--------------------+



Pada sesi ini, sebelum menghitung median kolom 'budget', baris dengan nilai null akan dihapus, sehingga datanya akurat. Didapat median dari kolom 'budget' adalag 5000000.

In [ ]:
# Menghapus baris dengan nilai null atau 0 pada kolom 'budget' dan menghitung median dari kolom 'budget'
from pyspark.sql.functions import isnan
df_temp = df.filter((df['budget'] != 0) & (df['budget'].isNotNull()) & (~isnan(df['budget'])))
median = df_temp.approxQuantile('budget',[0.5], 0.1)
print('The median of budget is ' + str(median))

The median of budget is [5000000.0]


Pada kode dibawah menghitung jumlah judul unik artinya, judul yang memiliki perbedaan dengan judul lainya, termasuk tidak ada kata diulang dan sama. Hasilnya, ada 41138 judul unik.

In [ ]:
# Menghitung jumlah judul unik dalam datafram
df.agg(countDistinct(col("title")).alias("count")).show()

+-----+
|count|
+-----+
|41138|
+-----+



Dari 41138 judul film yang unik, ditampilkan 10 judul film teratas.

In [ ]:
# Menampilkan 10 judul film unik
df.select('title').distinct().show(10, False)

+---------------------------------------------+
|title                                        |
+---------------------------------------------+
|The Corn Is Green                            |
|Meet The Browns - The Play                   |
|Morenita, El Escandalo                       |
|Father Takes a Wife                          |
|The Werewolf of Washington                   |
|My Wife Is a Gangster                        |
|Depeche Mode: Touring the Angel Live in Milan|
|A Woman Is a Woman                           |
|History Is Made at Night                     |
|Colombian Love                               |
+---------------------------------------------+
only showing top 10 rows



Dari judul-judul film unik diatas, akan diklasifikasikan berdasarkan tahun rilisnya. Hasilnya, dapat dilihat bahwa tahun 2007 adalah tahun paling banyak menghasilkan judul film unik yaitu sebanyak 1896 judul.

In [ ]:
# Menambahkan kolom 'release_year' yang berisi tahun dari tanggal rilis
df_temp=df.withColumn('release_year', year('release_date'))

In [ ]:
from pyspark.sql.functions import month,  dayofmonth, count_distinct

# Menambahkan kolom 'release month' yang berisi bulan dari tanggal rilis
df_temp = df_temp.withColumn('release month', month('release_date'))
# Menambahkan kolom 'release day' yang berisi hari dari tanggal rilis
df_temp = df_temp.withColumn('release day', dayofmonth('release_date'))
# Menambahkan kolom 'release year' yang berisi tahun dari tanggal rilis
df_temp = df_temp.withColumn('release year', year('release_date'))
# Mengelompokkan berdasarkan tahun rilis dan menghitung jumlah judul unik dalam setiap tahun
df_temp.groupBy("release_year").agg(countDistinct("title")).show(10, False)

+------------+---------------------+
|release_year|count(DISTINCT title)|
+------------+---------------------+
|1959        |271                  |
|1990        |496                  |
|1975        |365                  |
|1977        |415                  |
|1924        |19                   |
|2003        |1199                 |
|2007        |1896                 |
|2018        |4                    |
|1974        |434                  |
|2015        |13                   |
+------------+---------------------+
only showing top 10 rows



Disini kembali dilakukan filterisasi untuk menampilkan 10 baris pertama dalam dataframe yang judulnya dimulai dengan kata 'Meet'.

In [20]:
# Menampilkan 10 baris pertama dari dataframe yang judulnya dimulai dengan 'Meet'
df.filter(df['title'].like('Meet%')).show(10,False)

+-----+---------+----------+------------+-----------+--------------------------+
|id   |budget   |popularity|release_date|revenue    |title                     |
+-----+---------+----------+------------+-----------+--------------------------+
|43957|500000.0 |2.649     |2005-06-28  |1000000.0  |Meet The Browns - The Play|
|39997|0.0      |3.585     |1989-11-15  |0.0        |Meet the Hollowheads      |
|16710|0.0      |11.495    |2008-03-21  |4.1939392E7|Meet the Browns           |
|20430|0.0      |3.614     |2004-01-29  |0.0        |Meet Market               |
|76435|0.0      |1.775     |2011-03-31  |0.0        |Meet the In-Laws          |
|76516|5000000.0|4.05      |1990-11-08  |485772.0   |Meet the Applegates       |
|7278 |3.0E7    |11.116    |2008-01-24  |8.4646831E7|Meet the Spartans         |
|32574|0.0      |7.42      |1941-03-14  |0.0        |Meet John Doe             |
|40506|0.0      |4.814     |1997-01-31  |0.0        |Meet Wally Sparks         |
|40688|2.4E7    |6.848     |

Dilakukan kembali filtering yaitu memunculkan dataframe yang mengandung kata "one" yang berisi huruf dan angka.

In [23]:
df.filter(df['title'].rlike('\w*one')).show(10,False)

+-----+------+----------+------------+-------+------------------------+
|id   |budget|popularity|release_date|revenue|title                   |
+-----+------+----------+------------+-------+------------------------+
|43002|0.0   |5.62      |1962-05-24  |0.0    |Lonely Are the Brave    |
|43036|0.0   |5.438     |1960-08-30  |0.0    |Mill of the Stone Women |
|43051|0.0   |4.042     |1959-03-25  |0.0    |Al Capone               |
|43114|0.0   |5.703     |1958-08-01  |0.0    |Buchanan Rides Alone    |
|43250|0.0   |2.024     |1957-06-21  |0.0    |The Lonely Man          |
|43263|0.0   |8.947     |2010-05-09  |0.0    |Jesse Stone: No Remorse |
|43296|0.0   |4.123     |2008-03-05  |0.0    |Hormones                |
|43315|0.0   |3.455     |1955-10-17  |0.0    |A Man Alone             |
|43407|0.0   |3.155     |1965-02-24  |0.0    |None but the Brave      |
|43472|0.0   |1.357     |1946-01-01  |0.0    |Colonel Effingham's Raid|
+-----+------+----------+------------+-------+------------------

FIlering kembali dilakukan untuk menampilkan 20 baris data yang judulnya mengandung unsur 'ove' seperti 'rOVEre' dan 'lOVE'.

In [24]:
df.filter(df.title.contains('ove')).show()

+-----+------+------------------+------------+------------+--------------------+
|   id|budget|        popularity|release_date|     revenue|               title|
+-----+------+------------------+------------+------------+--------------------+
|43100|   0.0|             7.252|  1959-10-07|         0.0|General Della Rovere|
|43152|   0.0|             5.126|  2001-06-21|         0.0|      Love on a Diet|
|43191|   0.0|             4.921|  1952-08-29|         0.0|   Beware, My Lovely|
|43281|   0.0|             2.411|  1989-11-22|         0.0|   Love Without Pity|
|43343|   0.0|             3.174|  1953-12-25|         0.0|        Easy to Love|
|43347| 3.0E7|            14.863|  2010-11-22|1.02820008E8|  Love & Other Drugs|
|43362|   0.0|             1.705|  1952-02-23|         0.0|Love Is Better Th...|
|43363|   0.0|              2.02|  1952-05-29|         0.0|   Lovely to Look At|
|43395|   0.0|             4.758|  1950-11-10|         0.0| Two Weeks with Love|
|43455|   0.0|4.669000000000

Melakukan selecting untuk kolom atau variable yang dimulai dengan 're'. Didapatkan terdapat kolom release_date dan revenue. Output selecting berupa skema.

In [ ]:
# Memilih kolom yang nama kolomnya dimulai dengan 're'
df.select(df.colRegex("`re\w*`")).printSchema()

root
 |-- release_date: date (nullable = true)
 |-- revenue: float (nullable = true)



Melakukan selecting untuk kolom atau variable mengandung unsur 'e' didalamnya. Didapatkan terdapat kolom release_date, revenue dan title. Output selecting berupa skema.

In [ ]:
# Memilih kolom yang nama kolomnya mengandung huruf 'e'
df.select(df.colRegex("`\w*e`")).printSchema()

root
 |-- release_date: date (nullable = true)
 |-- revenue: float (nullable = true)
 |-- title: string (nullable = true)



## Creating New Columns

Pada sesi ini, dibuat kolom baru dengan tujuan untuk memuat informasi baru. Kolom baru pertama adalah kolom 'mean_popularity' atau rata-rata popularitas. Setelah dibuat kolom baru, dihitung varians atau ragam dalam kolom tersebut. Hasilnya, jumlah total variansnya sebanyak 1665668.6984730042 dengan varians populasinya sebesar 37.85782759382254.

In [25]:
# Menghitung rata-rata popularitas
mean_pop=df.agg({'popularity':'mean'}).collect()[0]['avg(popularity)']
# Menghitung jumlah observasi dalam dataframe
count_obs= df.count()

In [ ]:
# Menambahkan kolom baru 'mean_popularity' dengan nilai rata-rata popularitas
df=df.withColumn('mean_popularity',lit(mean_pop))

In [ ]:
from pyspark.sql.functions import pow, mean
# Menghitung rata-rata popularitas
mean_popularity = df.select(mean(df['popularity'])).collect()[0][0]
# Menambahkan kolom baru 'mean_popularity'
df = df.withColumn('mean_popularity', lit(mean_popularity))
# Menghitung total varians dari kolom 'popularity'
variance_sum = df.withColumn('variance', pow((df['popularity'] -
  df['mean_popularity']), 2)) \
                .agg({'variance': 'sum'}) \
                .collect()[0]['sum(variance)']
# Menampilkan total varians
print('Total variance:', variance_sum)

Total variance: 1665668.6984730042


In [ ]:
# Menghitung varians populasi
variance_population = variance_sum / count_obs
# Menampilkan varians populasi
print('Variance of population:', variance_population)

Variance of population: 37.85782759382254


Dilakukan proses lebeling bersadarkan kolom budget dan popularity dengan kondisi:

1. Lebeling budget \
    a) Jika budget kurang dari 10000000, maka beri lebel dalam kolom budget_cat 'Small' \
    b) Jika budget sama dengan 10000000, maka beri lebel dalam kolom budget_cat 'Medium' \
    c) Jika budget lebih dari 10000000, maka beri lebel dalam kolom budget_cat 'Big'

2. Lebeling popularity \
    a) Jika popularity kurang dari 3, maka beri lebel dalam kolom ratings 'Low' \
    b) Jika popularity kurang dari 5, maka beri lebel dalam kolom ratings 'Mid' \
    c) Jika popularity bernilai yang lain (lebih dari 5), maka beri lebel dalam kolom ratings 'High'.

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StructType, StructField, StringType

# Definisikan fungsi baru
def new_cols (budget, popularity):
  if budget < 10000000:
    budget_cat = "Small"
  elif budget < 100000000:
    budget_cat = "Medium"
  else:
    budget_cat = "Big"

  if popularity < 3:
    ratings = "Low"
  elif popularity < 5:
    ratings = "Mid"
  else:
    ratings = "High"

  return budget_cat, ratings

# Definisikan user-defined function (UDF)
def new_cols (budget, popularity):
  if budget < 10000000:
    budget_cat = "Small"
  elif budget < 100000000:
    budget_cat = "Medium"
  else:
    budget_cat = "Big"

  if popularity < 3:
      ratings = "Low"
  elif popularity < 5:
      ratings = "Mid"
  else:
      ratings = "High"
  return budget_cat, ratings

# Register UDF
udfB = udf (new_cols, StructType([
    StructField("budget_cat", StringType(), True),
    StructField("ratings", StringType(), True)
]))

# Select columns and apply the UDF
temp_df = df.select('id', 'budget', 'popularity').withColumn("newcat",
  udfB("budget", "popularity"))

# Unbundle the struct type columns into individual columns and drop the struct
df_with_newcols = temp_df.select('id', 'budget', 'popularity', temp_df.newcat.
  getItem('budget_cat').alias('budget_cat'),
                                 temp_df.newcat.getItem('ratings').
  alias('ratings')).drop('newcat')
# Menampilkan DataFrame yang sudah diubah
df_with_newcols.show(15, False)

+-----+---------+----------+----------+-------+
|id   |budget   |popularity|budget_cat|ratings|
+-----+---------+----------+----------+-------+
|43000|0.0      |2.503     |Small     |Low    |
|43001|0.0      |5.51      |Small     |High   |
|43002|0.0      |5.62      |Small     |High   |
|43003|0.0      |7.159     |Small     |High   |
|43004|500000.0 |3.988     |Small     |Mid    |
|43006|0.0      |3.194     |Small     |Mid    |
|43007|0.0      |2.689     |Small     |Low    |
|43008|0.0      |6.537     |Small     |High   |
|43010|0.0      |4.297     |Small     |Mid    |
|43011|0.0      |4.417     |Small     |Mid    |
|43012|7000000.0|4.722     |Small     |Mid    |
|43013|0.0      |2.543     |Small     |Low    |
|43014|0.0      |4.303     |Small     |Mid    |
|43015|0.0      |3.493     |Small     |Mid    |
|43016|0.0      |2.851     |Small     |Low    |
+-----+---------+----------+----------+-------+
only showing top 15 rows



Disini juga melakukan proses yang sama seperti diatas, hanya saja menggunakan modul yang berbeda yaitu 'when'. Kodenya juga lebih singkat dari kode diatas.

In [ ]:
#recreate the preceding columns using the when function
from pyspark.sql.functions import when

# Membuat DataFrame baru dengan kolom 'budget_cat' dan 'ratings' menggunakan when() function
df_with_newcols = df.select('id', 'budget', 'popularity').\
    withColumn('budget_cat', when (df ['budget'] < 10000000, 'Small').
  when (df ['budget'] < 100000000, 'Medium').otherwise('Big')).\
    withColumn('ratings', when (df ['popularity'] < 3, 'Low').
  when (df ['popularity'] <5, 'Mid').otherwise('High'))

# Menampilkan DataFrame yang sudah diubah
df_with_newcols.show(15, False)

+-----+---------+----------+----------+-------+
|id   |budget   |popularity|budget_cat|ratings|
+-----+---------+----------+----------+-------+
|43000|0.0      |2.503     |Small     |Low    |
|43001|0.0      |5.51      |Small     |High   |
|43002|0.0      |5.62      |Small     |High   |
|43003|0.0      |7.159     |Small     |High   |
|43004|500000.0 |3.988     |Small     |Mid    |
|43006|0.0      |3.194     |Small     |Mid    |
|43007|0.0      |2.689     |Small     |Low    |
|43008|0.0      |6.537     |Small     |High   |
|43010|0.0      |4.297     |Small     |Mid    |
|43011|0.0      |4.417     |Small     |Mid    |
|43012|7000000.0|4.722     |Small     |Mid    |
|43013|0.0      |2.543     |Small     |Low    |
|43014|0.0      |4.303     |Small     |Mid    |
|43015|0.0      |3.493     |Small     |Mid    |
|43016|0.0      |2.851     |Small     |Low    |
+-----+---------+----------+----------+-------+
only showing top 15 rows



## Deleting and Renamning Columns

Menghapus kolom budget_cat

In [ ]:
columns_to_drop=['budget_cat']
df_with_newcols=df_with_newcols.drop(*columns_to_drop)

Disini dilakukan kembali perubahan. Setelah kolom budget_cat dihapus, tersisa kolom ratings, namun kolom ratings akan diubah namanya menjadi film_ratings dengan fungsi `.withColumnRenamed`.

In [ ]:
# Menggunakan chaining untuk melakukan multiple transformations
df_with_newcols = df_with_newcols.withColumnRenamed('id', 'film_id').withColumnRenamed('ratings', 'film_ratings')

# Menampilkan DataFrame yang sudah diubah
df_with_newcols.show(15, False)

+-------+---------+----------+------------+
|film_id|budget   |popularity|film_ratings|
+-------+---------+----------+------------+
|43000  |0.0      |2.503     |Low         |
|43001  |0.0      |5.51      |High        |
|43002  |0.0      |5.62      |High        |
|43003  |0.0      |7.159     |High        |
|43004  |500000.0 |3.988     |Mid         |
|43006  |0.0      |3.194     |Mid         |
|43007  |0.0      |2.689     |Low         |
|43008  |0.0      |6.537     |High        |
|43010  |0.0      |4.297     |Mid         |
|43011  |0.0      |4.417     |Mid         |
|43012  |7000000.0|4.722     |Mid         |
|43013  |0.0      |2.543     |Low         |
|43014  |0.0      |4.303     |Mid         |
|43015  |0.0      |3.493     |Mid         |
|43016  |0.0      |2.851     |Low         |
+-------+---------+----------+------------+
only showing top 15 rows



Dilakan perubahan kembali dengan fungsi yang sama, yaitu merubah nama kolom popularity menjadi film_popularity.

In [ ]:
# change multiple column names, you try the following command:
# Mendefinisikan semua perubahan nama kolom dalam list
new_names = [('budget', 'film_budget'), ('popularity', 'film_popularity')]

# Mengubah nama kolom menggunakan loop
for old_name, new_name in new_names:
    df_with_newcols = df_with_newcols.withColumnRenamed(old_name, new_name)
# Menampilkan DataFrame yang sudah diubah
df_with_newcols.show(15, False)

+-------+-----------+---------------+------------+
|film_id|film_budget|film_popularity|film_ratings|
+-------+-----------+---------------+------------+
|43000  |0.0        |2.503          |Low         |
|43001  |0.0        |5.51           |High        |
|43002  |0.0        |5.62           |High        |
|43003  |0.0        |7.159          |High        |
|43004  |500000.0   |3.988          |Mid         |
|43006  |0.0        |3.194          |Mid         |
|43007  |0.0        |2.689          |Low         |
|43008  |0.0        |6.537          |High        |
|43010  |0.0        |4.297          |Mid         |
|43011  |0.0        |4.417          |Mid         |
|43012  |7000000.0  |4.722          |Mid         |
|43013  |0.0        |2.543          |Low         |
|43014  |0.0        |4.303          |Mid         |
|43015  |0.0        |3.493          |Mid         |
|43016  |0.0        |2.851          |Low         |
+-------+-----------+---------------+------------+
only showing top 15 rows



### **ANALISIS**

Dari percobaan ekspolarsi dengan Spark, dapat diambil berapa insight, yaitu:
1. Secara dataset, dataset ini terbilang buruk karena mengandung data yang tidak masuk akal. Contohnya ada adalah kolom budget dan revenue yang banyak sekali nilai 0. Contoh lainnya adalah ada film dengan budget 500000.0 namun revenuenya 0. Namun jika data dengan nilai NULL atau 0 dihapus, maka datasetnya akan tersisa sedikit.
2. Hasil ekplorasi ini dapat digunakan untuk menentukan strategi produksi. Terkhusus pada proses lebeling, kita dengan mudah mendapat informasi tentang budget dan rating filmnya. Produser film dapat mengambil judul film yang memiliki budget low namun rating high untuk dijadikan acuan strategi produksi hingga pemasaran.